In [1]:
from keras.models import Model, load_model
from keras import backend as K

def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth


print("Restoring model")
model = load_model("unet_model_v3.h5", custom_objects={'jaccard_distance_loss': jaccard_distance_loss})

Using TensorFlow backend.


Restoring model


## Submission

In [2]:
# %load mask_to_submission.py
#!/usr/bin/env python3

import os
import numpy as np
import matplotlib.image as mpimg
import re

foreground_threshold = 0.5 # percentage of pixels > 1 required to assign a foreground label to a patch

# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0
    
def predict_img(img):
    img1 = img[:400,:400]
    img2 = img[:400,-400:]
    img3 = img[-400:,:400]
    img4 = img[-400:,-400:]
    imgs = np.array([img1,img2,img3,img4])
    labels = model.predict(imgs).round()
    img_label = np.empty((608,608,1))
    img_label[-400:,-400:] = labels[3]
    img_label[-400:,:400] = labels[2]
    img_label[:400,-400:] = labels[1]
    img_label[:400,:400] = labels[0]
    return img_label
    
def mask_to_submission_strings(image_filename, i):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = i+1
    print("Predicting image {}".format(img_number))
    im = mpimg.imread(image_filename)
    im_label = predict_img(im)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im_label[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for i in range(len(image_filenames)):
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(image_filenames[i],i))


if __name__ == '__main__':
    submission_filename = 'submission.csv'
    image_filenames = []
    for i in range(1, 51):
        image_filename = 'test_set_images/test_' + '%d' % i +'/test_' + '%d' % i + '.png'
        print(image_filename)
        image_filenames.append(image_filename)
    masks_to_submission(submission_filename, *image_filenames)


test_set_images/test_1/test_1.png
test_set_images/test_2/test_2.png
test_set_images/test_3/test_3.png
test_set_images/test_4/test_4.png
test_set_images/test_5/test_5.png
test_set_images/test_6/test_6.png
test_set_images/test_7/test_7.png
test_set_images/test_8/test_8.png
test_set_images/test_9/test_9.png
test_set_images/test_10/test_10.png
test_set_images/test_11/test_11.png
test_set_images/test_12/test_12.png
test_set_images/test_13/test_13.png
test_set_images/test_14/test_14.png
test_set_images/test_15/test_15.png
test_set_images/test_16/test_16.png
test_set_images/test_17/test_17.png
test_set_images/test_18/test_18.png
test_set_images/test_19/test_19.png
test_set_images/test_20/test_20.png
test_set_images/test_21/test_21.png
test_set_images/test_22/test_22.png
test_set_images/test_23/test_23.png
test_set_images/test_24/test_24.png
test_set_images/test_25/test_25.png
test_set_images/test_26/test_26.png
test_set_images/test_27/test_27.png
test_set_images/test_28/test_28.png
test_set_i

In [20]:
# %load submission_to_mask.py
#!/usr/bin/python
import os
import sys
from PIL import Image
import math
import matplotlib.image as mpimg
import numpy as np

label_file = 'submission.csv'

h = 16
w = h
imgwidth = int(math.ceil((608.0/w))*w)
imgheight = int(math.ceil((608.0/h))*h)
nc = 3

# Convert an array of binary labels to a uint8
def binary_to_uint8(img):
    rimg = (img * 255).round().astype(np.uint8)
    return rimg

def reconstruct_from_labels(image_id):
    im = np.zeros((imgwidth, imgheight), dtype=np.uint8)
    f = open(label_file)
    lines = f.readlines()
    image_id_str = '%.3d_' % image_id
    for i in range(1, len(lines)):
        line = lines[i]
        if not image_id_str in line:
            continue

        tokens = line.split(',')
        id = tokens[0]
        prediction = int(tokens[1])
        tokens = id.split('_')
        i = int(tokens[1])
        j = int(tokens[2])

        je = min(j+w, imgwidth)
        ie = min(i+h, imgheight)
        if prediction == 0:
            adata = np.zeros((w,h))
        else:
            adata = np.ones((w,h))

        im[j:je, i:ie] = binary_to_uint8(adata)

    Image.fromarray(im).save('prediction_images/prediction_' + '%.3d' % image_id + '.png')

    return im

for i in range(1, 5):
    reconstruct_from_labels(i)
   
